In [4]:
import simpy
import numpy as np
from random import randint


In [5]:
# constants
half_time = 45
full_time = 90

pitch_dimensions = (105, 68)  # in m

max_outfield_players_on_team = 10
max_goalkeepers_on_team = 1

max_subs = 3
subs_on_bench = 7

yellow_cards = int
red_cards = int

if red_cards:
    max_outfield_players_on_team -= 1


# players follow fairly predictable paths
# receive ball -> run with ball -> short pass ball / long pass ball / clear ball / shoot / cross / infinitely hold ball
# goalkeepers are even simpler
# receive ball -> run with ball -> short pass ball / long pas ball / clear ball / infinitely hold ball

In [6]:
env = simpy.Environment()

# define some simpy resources
ball = simpy.Resource(env, capacity=1)


Trying to make player as a class for all possible actions that may occur

In [7]:
class Player(object):
    """
    Player starts game at time x, does various actions until game ends

    Attributes
    ----------

    w/ ball
    -------
    - run with ball
    - short pass
    - medium pass
    - long pass
    - infinitely hold ball
    - shoot
    - clear

    w/o ball
    --------
    - run without ball
        - (subcategories may be needed here)
    - tackle
        - gain possession
        - foul
    - pressure


    Player can either be with or without ball:

    """
    def __init__(self, env, player_name, team_name, position_name):

        # constants
        # environment
        self.env = env

        # names
        self.player_name = player_name
        self.team_name = team_name

        # starting position
        self.position_name = position_name
        self.now = env.now

        # anchor position defines the mean place where players are around
        self.anchor_position = 0

        # initialise starting position
        self.current_position = self.anchor_position
        self.new_position = self.anchor_position

        # initialise without ball
        self.has_ball = False

        # initialise number of goals from start
        self.player_goals = 0
        self.player_assists = 0


        # these team stats will be obtained from the team class
        self.team_goals = 0
        self.team_assists = 0

        self.opposition_goals = 0
        self.opposition_assists = 0

        # player stats out of 100
        self.prob_of_pass = 5
        self.prob_of_shoot = 3
        self.prob_of_losing_possession = 3
        self.prob_of_forward_run = 10
        self.prob_of_no_movement = 1
        self.prob_of_backwards_run = 8

    def play(self):
        if not self.sub:
            print(f"Starting Game for Player {self.player_name} in {self.team_name} at time {self.now}")
        else:
            print(f"Starting Game for Substitute {self.player_name} in {self.team_name} at time {self.now}")

        """
        While player not in possession:
            if ball is near:
                try:
                    tackle
                else:
                reset position
        """

        # while playing in game
        while True:
            if self.has_ball:
                # self.receive_ball
                print(f"{self.player_name} has ball at time {env.now}.")

                yield self.env.timeout(1)

    def actions_with_ball(self):
        # while player has ball
            while self.has_ball:

                """
                Here need to define different process:
                    - prob_of_pass
                    - prob_of_run
                """

                prob_of_pass = randint(0, 100)
                prob_of_shoot = randint(0, 100)

                if prob_of_pass <= 10: # 10 times out of 100 player will pass
                    print(f"{self.player_name} passed at time: {env.now}!")
                    self.has_ball = False
                    self.current_position = self.anchor_position

                elif prob_of_shoot <= 5:
                    print(f"{self.player_name} shoots...")
                    if prob_of_shoot == randint(1, 5):
                        print("He Scores!!!")
                        self.team_goals += 1
                        self.player_goals += 1
                        self.current_position = self.anchor_position
                    else:
                        print(f"He misses. Opposition has ball")
                    self.has_ball = False

                else:
                    # run with ball
                    self.current_position += randint(-175, 250) / 100  # at first approx: go forward or back by up to 2.5 meters
                    self.new_position = abs(self.current_position - self.anchor_position)
                    if self.current_position < self.anchor_position:
                        print(f"Moved backward by {round(abs(self.current_position - self.anchor_position), 2)}")
                    elif self.current_position > self.anchor_position:
                        print(f"Moved forward by {round(abs(self.current_position - self.anchor_position), 2)}")
                    else:
                        print(f"At Anchor Position")
                yield self.env.timeout(1)
            else:
                # print(f"Time is {env.now}")
                prob_of_getting_ball = randint(0, 100)
                if prob_of_getting_ball <= 5:  # 5 times out of 100 player gets ball:
                    self.has_ball = True
                yield self.env.timeout(1)



    def receive_ball(self):
        """
        Receive ball occurs in a few situations:

        - Player is passed to from friendly player:
        - Player is passed to from opposition player:
        - Player intercepts ball from opposition player:
        - Player receives ball from throw-in
        -

        Returns:

        """
        while True:
            self.has_ball = True
            print(f"{self.player_name} now in possession")

In [8]:
class Pitch(object):
    def __init__(self, pitch_dimensions, goal_size, grass_type):

        self.pitch_dimensions = pitch_dimensions
        self.goal_size = goal_size
        self.grass_type = grass_type

In [9]:
class Team(object):
    def __init__(self, players, formation, kick_off_player):

        # load in 11 starting player object
        self.players = players

        # formation
        self.formation = formation

        if kick_off_player:
            self.kick_off_player = kick_off_player
        else:
            self.kick_off_player = players["ST"]

    def get_kick_off_player(self):
        return self.kick_off_player

In [56]:
class Match(object):
    def __init__(self, environment, home_team, away_team, player):

        """
        All play actions will occur in match. All stats are stored in players, match and pitch

        Args:
            home_team:
            away_team:
        """

        # match simpy environment
        self.environment = environment

        # teams as team classes
        self.home_team = home_team
        self.away_team = away_team

        # initial scores
        self.home_goals = 0
        self.away_goals = 0

        # test player
        self.player = player

        self.score = f"{self.home_goals}:{self.away_goals}"

        # time
        self.time = 0

        # start game
        self.start = self.environment.process(self.start_game())

        # kick of players
        # self.home_kick_off_player = home_team.kick_off_player
        # self.away_kick_off_player = away_team.kick_off_player

        # player in possession at start (assuming home team kicks off)
        # self.player_in_possession = self.home_kick_off_player

    def start_game(self):
        print("-" * 50 + "\n")
        print("Simulated game using simpy \n")
        print("-" * 50 + "\n")

        print(f"{self.home_team} vs {self.away_team}")
        print("-" * 50 + "\n")
        print(f"Games starts!")

        player_in_possession = self.player # self.home_team.get_kick_off_player()

        print(f"{self.player.player_name} starts the game off.")
        print(f"    Time: {self.time}")
        while True:

            '''
            The cycle from here is pretty simple

            -> player x has ball at time t

            -> movement, pass, shoot or tackled events happen

                -> if movement, player keeps ball
                -> else, player_in_possession changes

            -> repeat until player no longer has ball

            '''
            yield self.environment.process(self.player_action())



    def player_action(self):
        """
        defines any given action a player can do
        Returns:

        """
        while True:

            """
            Here need to define different process:
                - prob_of_pass
                - prob_of_run
            """
            self.time += 1
            action = self.get_action()

            if action == "pass": # 10 times out of 100 player will pass
                print(f"{self.player.player_name} passed at time: {env.now}!")
                self.player.current_position = self.player.anchor_position
                self.player.has_ball = False

            elif action == "shoot":
                print(f"{self.player.player_name} shoots...")

                if self.player.prob_of_shoot == randint(1, 5):
                    print("He Scores!!!")
                    self.player.team_goals += 1
                    self.player.player_goals += 1
                    self.player.current_position = self.player.anchor_position
                else:
                    print(f"He misses. Opposition has ball")
                self.player.has_ball = False

            else:

                # run with ball
                self.player.current_position += randint(-175, 250) / 100  # at first approx: go forward or back by up to 2.5 meters
                self.player.new_position = abs(self.player.current_position - self.player.anchor_position)

                # determine backwards or forwards movement
                if self.player.current_position < self.player.anchor_position:
                    print(f"Moved backward by {round(abs(self.player.current_position - self.player.anchor_position), 2)}")
                elif self.player.current_position > self.player.anchor_position:
                    print(f"Moved forward by {round(abs(self.player.current_position - self.player.anchor_position), 2)}")
                else:
                    print(f"At Anchor Position")

            print(f"    Time: {self.time}")
            yield self.environment.timeout(1)



    def get_action(self):
        from numpy.random import choice
        """

        Args:
            player (Player Object): player containing all probability stats

        Returns:
            action (str): action for player to do
        """


        # probabilities of different actions happening (note that at 1st approx these actions are independent of other players or position on pitch)
        probabilities = {"pass": self.player.prob_of_pass,  # out of 100?
        "shoot": self.player.prob_of_shoot,  # + self.shooting_falloff(player.position)  # TODO: need to work out a general falloff for shots going in
        "lose_ball": self.player.prob_of_losing_possession,  # measure only of being tackled not of wayward pass or shot
        "run_forward": self.player.prob_of_forward_run,
        "no_run": self.player.prob_of_no_movement,
        "run_back": self.player.prob_of_backwards_run}

        # normalise to percentage
        normalised_probabilities = {key: val for key, val in zip(probabilities.keys(), [prob / sum(probabilities.values()) for prob in probabilities.values()])}

        action = choice(list(normalised_probabilities.keys()), 1, p=list(normalised_probabilities.values()), replace=True)

        return action


In [57]:
env = simpy.Environment()
bobson_dugnutt = Player(env, player_name="Bobson Dugnutt", team_name="Pumblechook FC", position_name="ST")
match = Match(env, bobson_dugnutt.team_name, "AntiFC", bobson_dugnutt)
env.run(90)









--------------------------------------------------

Simulated game using simpy 

--------------------------------------------------

Pumblechook FC vs AntiFC
--------------------------------------------------

Games starts!
Bobson Dugnutt starts the game off.
    Time: 0
Moved backward by 0.66
    Time: 1
Moved backward by 0.72
    Time: 2
Bobson Dugnutt passed at time: 2!
    Time: 3
Moved backward by 1.61
    Time: 4
Moved forward by 0.11
    Time: 5
Moved forward by 0.49
    Time: 6
Moved forward by 1.68
    Time: 7
Bobson Dugnutt shoots...
He misses. Opposition has ball
    Time: 8
Bobson Dugnutt passed at time: 8!
    Time: 9
Moved backward by 1.2
    Time: 10
Bobson Dugnutt shoots...
He misses. Opposition has ball
    Time: 11
Moved backward by 2.93
    Time: 12
Moved backward by 4.16
    Time: 13
Moved backward by 2.07
    Time: 14
Bobson Dugnutt passed at time: 14!
    Time: 15
Bobson Dugnutt shoots...
He misses. Opposition has ball
    Time: 16
Bobson Dugnutt shoots...
He miss